# MoBioFP - Fingerphoto Recognition (Fingertip Semantic Segmentation)

In [ ]:
import cv2

from mobiofp.segmentation import Segment
from mobiofp.utils import (
    fingertip_enhancement,
    fingerprint_mapping,
    fingerprint_enhancement,
    fingertip_thresholding,
)

from shared import read_images, show_images

## Define global constants

In [ ]:
SAMPLE_DIR = "../data/raw/samples"
PROCESSED_DIR = "../data/processed/samples/segmentation"

# Assume the model is already downloaded and placed in the models directory.
# Use one of the following models based on your system architecture.
# MODEL_CHECKPOINT = "../models/fingertip-seg-amd64.h5" # For AMD64
MODEL_CHECKPOINT = "../models/fingertip-seg-arm64.h5"  # For ARM64

## Read sample images

In [ ]:
images, images_titles = read_images(SAMPLE_DIR)
show_images(images, images_titles, fig_size=15, sup_title="Sample Fingerphoto Images")

## Fingertip segmentation using U-Net pre-trained model

In [ ]:
model = Segment()
model.load(MODEL_CHECKPOINT)

results = [model.predict(image) for image in images]

bbox_coords = []
fingertip_images = []
fingertip_masks = []

for image, result in zip(images, results):
    bbox = model.extract_roi(result)
    bbox_coords.append(bbox)

    fingertip = model.crop_image(image, bbox)
    fingertip_images.append(fingertip)

    fingertip_mask = model.crop_image(result, bbox)
    fingertip_masks.append(fingertip_mask)

In [ ]:
show_images(fingertip_images, images_titles, sup_title="Fingertip Images")
show_images(fingertip_masks, images_titles, sup_title="Fingertip Masks")

## Fingertip Enhancement

In [ ]:
gray_images = [cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) for image in fingertip_images]
show_images(gray_images, images_titles, cmap="gray", sup_title="Grayscale Fingertip Images")

In [ ]:
fingertip_enhanced_images = [fingertip_enhancement(image) for image in gray_images]
show_images(fingertip_enhanced_images, images_titles, sup_title="Fingertip Enhanced Images")

## Fingertip Binarization

In [ ]:
fingertip_thresh_images = [
    cv2.bitwise_and(image, image, mask=mask)
    for image, mask in zip(fingertip_enhanced_images, fingertip_masks)
]
fingertip_thresh_images = [
    fingertip_thresholding(image, blockSize=19) for image in fingertip_thresh_images
]
show_images(
    fingertip_thresh_images, images_titles, cmap="gray", sup_title="Fingertip Thresholded Images"
)

## Fingertip to Fingerprint Conversion

The function `fingerprint_mapping()` takes an fingertip-enhanced and converts it into a fingerprint image.
It does this by:

- Apply Adaptive Mean Thresholding
- Resizing the image.
- Calculating the local gradient of the image using Sobel filters.
- Calculating the orientation of the ridges in the fingerprint.
- Extracting a region of the image and smoothing it to reduce noise.
- Calculating the x-signature of the region and finding its local maxima to estimate the ridge period.
- Creating a bank of Gabor filters with different orientations.
- Filtering the image with each filter in the bank.
- Assembling the final result by taking the corresponding convolution result for each pixel based on the closest orientation in the Gabor bank.

In [ ]:
fingerprints, fingerprint_titles = [], []

for image, title in zip(fingertip_thresh_images, images_titles):
    fingerprint = fingerprint_mapping(image)
    if fingerprint is not None:
        fingerprints.append(fingerprint)
        fingerprint_titles.append(title)

show_images(fingerprints, fingerprint_titles, sup_title="Fingerprint Images")

In [ ]:
fingerprint_enhanced_images = [fingerprint_enhancement(fingerprint) for fingerprint in fingerprints]

show_images(
    fingerprint_enhanced_images,
    fingerprint_titles,
    sup_title="Fingerprint Enhanced Images",
)

## Thinning

In [ ]:
fingerprint_thinning_images = [
    cv2.ximgproc.thinning(image) for image in fingerprint_enhanced_images
]

show_images(
    fingerprint_thinning_images,
    fingerprint_titles,
    sup_title="Fingerprint Thinned Images",
)